In [13]:
from pydantic import BaseModel
import asyncio

In [2]:
class TestMsg(BaseModel):
    msg: str

In [11]:
msg = TestMsg(msg="hello world")

In [4]:
import time
from aiokafka import AIOKafkaProducer, AIOKafkaConsumer

from fastkafka.testing import Tester, ApacheKafkaBroker

# Benchmark producing speed

## AIOKafka

In [5]:
with ApacheKafkaBroker(topics=["test_topic"], apply_nest_asyncio=True) as broker:
    producer = AIOKafkaProducer(bootstrap_servers=broker)
    await producer.start()
    t0 = time.time()
    for _ in range(1_000_000):
        await producer.send("test_topic", msg.json().encode("utf-8"))
    await producer.flush()
    t1 = time.time()
    await producer.stop()
    
print(f" Avg AIOKafkaProducer producing speed: {1_000_000/(t1-t0)} msg/s")

[INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): entering...
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): (<_UnixSelectorEventLoop running=True closed=False debug=False>) is already running!
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): calling nest_asyncio.apply()
[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
[INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._testing.apache_kafka_broker: <class 'fastkafka.testing.ApacheKafkaBroker'>.sta

## FastKafka

In [6]:
from fastkafka import FastKafka

In [7]:
with ApacheKafkaBroker(topics=["test_topic"], apply_nest_asyncio=True) as broker:
    app = FastKafka(kafka_brokers={
        "localhost": {
            "url": "127.0.0.1",
            "port": "9092"
        }
    })
    
    @app.benchmark()
    @app.produces()
    async def to_test_topic(msg: TestMsg) -> TestMsg:
        return msg
    
    async with app:
        for _ in range(1_000_000):
            await to_test_topic(msg)

[INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): entering...
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): (<_UnixSelectorEventLoop running=True closed=False debug=False>) is already running!
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): calling nest_asyncio.apply()
[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
[INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._testing.apache_kafka_broker: <class 'fastkafka.testing.ApacheKafkaBroker'>.start(): returning 127.0.0.1:9092
[INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): exited.
[INFO] fastkafka._application.app: _create_producer() : creat

# Benchmark consuming speed

## AIOKafka

In [25]:
with ApacheKafkaBroker(topics=["test_topic"], apply_nest_asyncio=True) as broker:
    producer = AIOKafkaProducer(bootstrap_servers=broker)
    await producer.start()
    for _ in range(1_000_000):
        await producer.send("test_topic", msg.json().encode("utf-8"))
    await producer.flush()
    await producer.stop()

    consumer = AIOKafkaConsumer(
        "test_topic", bootstrap_servers=broker, auto_offset_reset="earliest"
    )

    i = 0

    await consumer.start()
    t0 = time.time()
    try:
        async for _ in consumer:
            i += 1
            if i == 1_000_000:
                break
            
    finally:
        await consumer.stop()
    t1 = time.time()

print(f" Avg AIOKafkaProducer consuming speed: {1_000_000/(t1-t0)} msg/s")

[INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): entering...
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): (<_UnixSelectorEventLoop running=True closed=False debug=False>) is already running!
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): calling nest_asyncio.apply()
[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
[INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._testing.apache_kafka_broker: <class 'fastkafka.testing.ApacheKafkaBroker'>.start(): returning 127.0.0.1:9092
[INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): exited.
[INFO] aiokafka.consumer.subscription_state: Updating subscri

## FastKafka

In [24]:
with ApacheKafkaBroker(topics=["test_topic"], apply_nest_asyncio=True) as broker:
    producer = AIOKafkaProducer(bootstrap_servers=broker)
    await producer.start()
    t0 = time.time()
    for _ in range(1_000_000):
        await producer.send("test_topic", msg.json().encode("utf-8"))
    await producer.flush()
    t1 = time.time()
    await producer.stop()
    
    app = FastKafka(kafka_brokers={
        "localhost": {
            "url": "127.0.0.1",
            "port": "9092"
        }
    })
    
    i = 0
    
    @app.consumes(auto_offset_reset="earliest")
    @app.benchmark()
    async def on_test_topic(msg: TestMsg):
        i += 1
        if i != 1_000_000:
    
    async with app:
        await asyncio.sleep(5)

[INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): entering...
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): (<_UnixSelectorEventLoop running=True closed=False debug=False>) is already running!
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): calling nest_asyncio.apply()
[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
[INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._testing.apache_kafka_broker: <class 'fastkafka.testing.ApacheKafkaBroker'>.start(): returning 127.0.0.1:9092
[INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): exited.
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka

In [22]:
i

1000000